In [21]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import pandas as pd
import numpy as np

import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Kkma, Komoran, Okt, Hannanum  #Mecab은 윈도우에서 작동 불가능

from konlpy import tag
from konlpy.corpus import kolaw
# from konlpy.utils import csvwrite, pprint

from nltk.tokenize import sent_tokenize

In [100]:
Lyrics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/Lyrics_final.csv')

In [24]:
# 행복, 희망 => 1(긍정), 슬픔, 분노 => 0(부정) 으로 이진분류
def rating_to_label(label):
    if label == 0 or label == 3:
        return 1
    else:
        return 0

Lyrics['y'] = Lyrics['label'].apply(lambda x: rating_to_label(x))
Lyrics_pos = Lyrics[Lyrics['y'] == 1]
Lyrics_neg = Lyrics[Lyrics['y'] == 0]

In [101]:
kosac = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/odatt/polarity.csv")

### 태깅

In [102]:
okt = Okt() 
kkma = Kkma() #kosac에 알맞는 데이터 값
komoran = Komoran()
hannanum = Hannanum()

kkma 이용

In [103]:
data_pos = Lyrics_pos['Lyrics'].to_string()
data_neg = Lyrics_neg['Lyrics'].to_string()

kkma_pos = kkma.pos(data_pos)
kkma_neg = kkma.pos(data_neg)


# 형태소 필터링 

# kkma_text = ['NNG', 'NNP', 'NNB','VV','VA','MAG','IC','NR']
# clean_words_pos = []
# for i in kkma_pos:
#   for j in kkma_text:
#     if j in i:
#       clean_words_pos.append(i)

# clean_words_neg = []
# for i in kkma_neg:
#   for j in kkma_text:
#     if j in i:
#       clean_words_neg.append(i)


In [110]:
# 단어+형태소 합치기 

kkma_pos_list = []
kkma_neg_list = []

for i in range(len(kkma_pos)):
  pos_text = kkma_pos[i][0]+'/'+kkma_pos[i][1]
  kkma_pos_list.append(pos_text)
  i += 1

for i in range(len(kkma_neg)):
  neg_text = kkma_neg[i][0]+'/'+kkma_neg[i][1]
  kkma_neg_list.append(neg_text)
  i += 1


In [218]:
# 형태소분석이 된 긍정가사를 가사별로 리스트화하여 dataframe으로 정리
text_pos = ''.join(str(i) for i in kkma_pos_list)
text_neg = ' '.join(str(i) for i in kkma_neg_list)

text_pos_list = []
numList_pos = []

for i in range(1000):
  if(text_pos.find(f'{i}') > -1):
    num_pos = text_pos.find(f'{i}')
    numList_pos.append(num_pos)

for i in range(len(numList_pos)):
  text = ""
  if(i == len(numList_pos)-1):
    text = text_pos[int(numList_pos[i]):len(text_pos)]
  else:
    text = text_pos[int(numList_pos[i]):int(numList_pos[i+1])]
  text_pos_list.append(text)

aa = text_pos_list[0]
text_pos_list.append(aa)

pos = pd.DataFrame(text_pos_list)
pos = pos.drop(0, axis = 0 )
pos.columns = ['word']

# 형태소분석이 된 부정가사를 가사별로 리스트화하여 dataframe으로 정리
text_neg_list = []
numList_neg = []

for i in range(196,1000):
  if(text_neg.find(f'{i}') > -1):
    num_neg = text_neg.find(f'{i}')
    numList_neg.append(num_neg)

for i in range(len(numList_neg)):
  text_1 = ""
  if(i == len(numList_neg)-1):
    text_1 = text_neg[numList_neg[i]:len(text_neg)]
  else:
    text_1 = text_neg[numList_neg[i]:int(numList_neg[i+1])]
  text_neg_list.append(text_1)


dd = text_neg_list[0]
text_neg_list.append(dd)

neg = pd.DataFrame(text_neg_list)
neg = neg.drop(0, axis = 0 )
neg.columns = ['word']

####okt 이용

In [ ]:
okt = Okt()

In [ ]:
data_pos = Lyrics_pos['Lyrics'].to_string()
data_neg = Lyrics_neg['Lyrics'].to_string()

In [ ]:
pos_pos = okt.pos(data_pos)
list_pos = []
list_neg = []
for i in range(len(pos_pos)):
  pos_text = pos_pos[i][0]+'/'+pos_pos[i][1]
  list_pos.append(pos_text)
  i += 1
  
pos_neg = okt.pos(data_neg)
for i in range(len(pos_neg)):
  neg_text = pos_neg[i][0]+'/'+pos_neg[i][1]
  list_neg.append(neg_text)
  i += 1

In [ ]:
# 형태소분석이 된 긍정가사를 가사별로 리스트화하여 dataframe으로 정리
text = ' '.join(str(i) for i in kkma_pos)
text_pos = text.split('/Number')

pos = pd.DataFrame(text_pos)
pos = pos.drop(0, axis = 0 )
pos.columns = ['word']

# 형태소분석이 된 부정가사를 가사별로 리스트화하여 dataframe으로 정리
text = ' '.join(str(i) for i in kkma_neg)
text_neg = text.split('/Number')

neg = pd.DataFrame(text_neg)
neg = neg.drop(0, axis = 0 )
neg.columns = ['word']

### KoSac 코드 (kkma 이용)

가사별로 형태소분석 list로 나눠주기 => 이후 dataframe화 함

:

In [219]:
# kosac을 POS 와 NEG로 분류
kosac_pos = kosac[kosac['max.value'] == 'POS']
kosac_neg = kosac[kosac['max.value'] == 'NEG']

In [220]:
def LyricsWordCount(arrList, emotion):
    list = []
    i=1

    # 감성 점수 구하기
    for Lyrics in arrList:
        neg_count = 0       # 부정 점수
        pos_count = 0       # 긍정 점수
        num = 0             # 감성 점수
        
        # 긍정 점수 구하기
        for pos_text in kosac_pos['ngram']:
            pos_find = Lyrics.find(pos_text)
            if(pos_find > -1):
                pos_count += 1
            else: pos_count += 0


        # 부정 점수 구하기
        for neg_text in kosac_neg['ngram']:
            neg_find = Lyrics.find(neg_text)
            if(neg_find > -1):
                neg_count += 1
            else: pos_count += 0

        # 감성 점수 공식
        if emotion == '긍정':
            num = pos_count - neg_count
        elif emotion == '부정':
            num = neg_count - pos_count

        # 감성 점수 공식
        if emotion == '긍정':
          if pos_count == 0 or neg_count == 0:
            num = 0
          else:
            num = pos_count/(pos_count + neg_count)
            
        elif emotion == '부정':
          if pos_count == 0 or neg_count == 0:
            num = 0
          else:
            num = neg_count/(pos_count + neg_count)

        # 긍, 부정 결과
        if emotion == '긍정':
          if num >= 0.5:
            result = '긍정'
          elif num < 0.5:
            result = '부정'
        
        if emotion == '부정':
          if num >= 0.5:
            result = '부정'
          elif num < 0.5:
            result = '긍정'


        srt = f"{i}번 노래 결과 : 긍정 : {pos_count}, 부정 : {neg_count} , 감성 점수 : {round(num,3)}  {result}"
        i = i + 1

        list.append(srt)
    return list

pos_result = LyricsWordCount(pos['word'], "긍정")
neg_result = LyricsWordCount(neg['word'], "부정")

print('긍정 노래')
for pos in pos_result:
    print(pos)

print('부정 노래')
for neg in neg_result:
    print(neg)

긍정 노래
1번 노래 결과 : 긍정 : 6, 부정 : 9 , 감성 점수 : 0.4  부정
2번 노래 결과 : 긍정 : 9, 부정 : 6 , 감성 점수 : 0.6  긍정
3번 노래 결과 : 긍정 : 5, 부정 : 7 , 감성 점수 : 0.417  부정
4번 노래 결과 : 긍정 : 6, 부정 : 5 , 감성 점수 : 0.545  긍정
5번 노래 결과 : 긍정 : 8, 부정 : 4 , 감성 점수 : 0.667  긍정
6번 노래 결과 : 긍정 : 7, 부정 : 11 , 감성 점수 : 0.389  부정
7번 노래 결과 : 긍정 : 8, 부정 : 6 , 감성 점수 : 0.571  긍정
8번 노래 결과 : 긍정 : 4, 부정 : 10 , 감성 점수 : 0.286  부정
9번 노래 결과 : 긍정 : 9, 부정 : 6 , 감성 점수 : 0.6  긍정
10번 노래 결과 : 긍정 : 6, 부정 : 2 , 감성 점수 : 0.75  긍정
11번 노래 결과 : 긍정 : 5, 부정 : 9 , 감성 점수 : 0.357  부정
12번 노래 결과 : 긍정 : 4, 부정 : 10 , 감성 점수 : 0.286  부정
13번 노래 결과 : 긍정 : 5, 부정 : 4 , 감성 점수 : 0.556  긍정
14번 노래 결과 : 긍정 : 5, 부정 : 8 , 감성 점수 : 0.385  부정
15번 노래 결과 : 긍정 : 10, 부정 : 3 , 감성 점수 : 0.769  긍정
16번 노래 결과 : 긍정 : 8, 부정 : 6 , 감성 점수 : 0.571  긍정
17번 노래 결과 : 긍정 : 7, 부정 : 5 , 감성 점수 : 0.583  긍정
18번 노래 결과 : 긍정 : 7, 부정 : 9 , 감성 점수 : 0.438  부정
19번 노래 결과 : 긍정 : 6, 부정 : 8 , 감성 점수 : 0.429  부정
20번 노래 결과 : 긍정 : 5, 부정 : 6 , 감성 점수 : 0.455  부정
21번 노래 결과 : 긍정 : 1, 부정 : 4 , 감성 점수 : 0.2  부정
22번 노래 결과 : 긍정 : 6, 부

In [221]:
def Total_pos(list) :
    count = 0
    for i in list:
        if(i[len(i)-2:len(i)] == '긍정'):
            count = count + 1

    return count        

sum = Total_pos(pos_result)
print('긍정 정확도: ', sum/len(Lyrics_pos))

def Total_neg(list) :
    count = 0
    for i in list:
        if(i[len(i)-2:len(i)] == '부정'):
            count = count + 1

    return count        

sum = Total_neg(pos_result)
print('부정 정확도: ', sum/len(Lyrics_pos))

긍정 정확도:  0.5404624277456648
부정 정확도:  0.4595375722543353


In [222]:
# 긍정정.csv 파일 저장
import csv
with open('kosac_pos.csv', 'w', newline= '\n') as f:
  writer = csv.writer(f)
  writer.writerow(pos_result)

In [223]:
# 부정.csv 파일 저장
import csv
with open('kosac_neg.csv', 'w', newline= '\n') as f:
  writer = csv.writer(f)
  writer.writerow(neg_result)